# Exercise 3 ( Warmup with multidimensional stencils )

In this exercise we want to solve:
$$
\left\{ \begin{array}{rll} - \Delta u &= f  \qquad &\text{in} \; \; \Omega := (0,1)^2 \subset \mathbb{R}^2,\\
u & = 0 \qquad &\text{on} \;\; \partial \Omega,
\end{array} \right.
$$

where we define $u$ and $f$ as:
$$
u(x,y) := \sin( 3 \pi x ) \sin( \pi y ) \qquad f(x,y) := 10 \pi^2 \sin( 3 \pi x ) \sin( \pi y )
$$


### a)

Take a look at the ```kron``` function from scipy.sparse. Use the function to create the discretized Laplace operator in arbritrary dimensions. For that write a function ```laplace_matrix_nd```, which takes a tuple $n$ as an input and returns the matrix $L_h$ that represents the laplace operator for a discretization with a grid containing $n_i$ grid points in each dimension. Inspect the resulting matrix for $n \in \{ (4,5), (4,5,6),(4,5,6,7) \}$ with ```spy```.

<b>Hint:</b> Write a recursive function.

In [1]:
import numpy as np
import scipy as sp
from scipy.sparse import kron, eye, diags, csc_matrix
import scipy.sparse.linalg

import matplotlib.pyplot as plt
%matplotlib qt

def laplace_matrix_nd( n ):
    if len(n) == 1:
        h = 1/( n[0] + 1 ) #step size for n1
        return -diags([-2*np.ones(n[0]),np.ones(n[0]),np.ones(n[0])],[0,-1,1]) / (h**2)  #L matrix for first dimension n1
    else:
        h = 1 / (n[-1]+1) #step size for i 
        d = np.prod(n[:-1]) 
        I = ( eye( n[-1] * d , n[-1] * d , -d ) + eye( n[-1] * d, n[-1] * d ,d ) ) / (h**2) # In for upper and lower diagonal
        T = kron( diags( np.ones(n[-1]) ,0) , laplace_matrix_nd( n[:-1] ) ) + 2 * diags( np.ones(n[-1] * d)  ,0 )/ (h**2) # Tn for diagonal
        return T - I
        
plt.figure(figsize=(12,12))
plt.subplots_adjust(hspace=0.3,wspace=0.3)
for n in [(4,), (4,5), (4,5,6), (4,5,6,7)]:
    plt.subplot(2,2, [(4,), (4,5), (4,5,6), (4,5,6,7)].index(n)+1 )
    plt.spy(laplace_matrix_nd( n ).toarray()) 

### b)

For $n= (20,30)$ create the corresponding grid and then create the right hand side $f_h$. Solve the BVP and plot the approximated solution $u_h$ in a surface plot right next to the analytic solution $u$ with $n=(100,100)$.

In [3]:
def f(x,y):
    return 10 * np.pi ** 2 * np.sin( 3 * np.pi * x ) * np.sin( np.pi * y)

def u(x,y):
    return np.sin( 3 * np.pi * x ) * np.sin( np.pi * y)

# Your code
n = (100,110)

#Construct f_h
x = np.linspace(0,1,n[0]+2)[1:-1] # x grid
y = np.linspace(0,1,n[1]+2)[1:-1] # y grid

fh = []
for i in y:
    for j in x:
        fh.append( f(j,i) ) # Construct discretized f_h

Lh = csc_matrix(laplace_matrix_nd(n)) 
uh = scipy.sparse.linalg.spsolve(Lh,fh) #Solve L_h * u_h = f_h

Uh = np.zeros(n) #Construct matrix u_h
for i in range(n[0]):
    for j in range(n[1]):
        Uh[i][j] = uh[j*n[0]+i]  

# Plot
fig = plt.figure(figsize=plt.figaspect(0.4))
x,y = np.meshgrid(x,y)
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_surface(x, y, Uh.T,cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('discretized u_h')
ax.set_title('3D plot of discretized u_h')

Ua = u(x,y)
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(x, y, Ua,cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('analytic u')
ax.set_title('3D plot of analytic u')

Text(0.5, 0.92, '3D plot of analytic u')

### c)

Compute the error between $u_h$ and $u$ in the maximum norm. Create a log-log plot of the error against the size, e.g the number of variables for the linear system, for the following two different strategies to choose $n_1$ and $n_2$:

- For ten approximately logarithmic evenly spaced $n_1$ between 10 and 1000 and fixed $n_2 = 100$

- For ten approximately logarithmic evenly spaced $n_1 = n_2$ between 30 and 300.

Plot both graphs in the same figure.

<b>Note:</b> Make sure $n_i$ is always an integer.

In [ ]:
def error( x_h, u_h ):
    
    u_a = u(np.array(x_h).T[0],np.array(x_h).T[1]) # analytic solution of the PDE
    error = max( abs( u_a - u_h ) ) # error: max( |ua - uh| )
    
    return error

# Your code
# Case 1
n1 = np.round(np.logspace(1,3,10)) # number of grid points for the 1st dimension
n2 = 100                           # number of grid points for the 2nd dimension
error_list = []                    # the list to store the error in different n1
y = np.linspace(0, 1, n2+2)[1:-1]  # grid for the 2nd dimension y
for k in n1:
    x = np.linspace(0, 1, int(k) +2 )[1:-1]  # x grid
    fh = []   # list for discretized f
    xh = []   # list for discretized space
    for i in y:
        for j in x:
            fh.append( f(j,i) )
            xh.append( (j,i) )
    uh = scipy.sparse.linalg.spsolve(laplace_matrix_nd((int(k),n2)),fh) #Solve L_h * u_h = f_h
    error_list.append( error(xh,uh) )

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(8,4,forward=True)
ax[0].loglog(n1,error_list)
ax[0].set_title('error vs n1=10,...1000')
ax[0].set_xlabel('n1')
ax[0].set_ylabel('error')

# Case 2
n1 = np.round(3*np.logspace(1,2,10)) # number of grid points for the 1st dimension
error_list2 = []
for k in n1:
    x = np.linspace(0, 1, int(k) +2 )[1:-1]  # x grid
    y = x # grid for y
    fh = []
    xh = []
    for i in y:
        for j in x:
            fh.append( f(j,i) )
            xh.append( (j,i) )
    uh = scipy.sparse.linalg.spsolve(laplace_matrix_nd((int(k),int(k))),fh) #Solve L_h * u_h = f_h
    error_list2.append( error(xh,uh) )

ax[1].loglog(n1,error_list2)
ax[1].set_title('error vs n1,n2=30,...300')
ax[1].set_xlabel('n1(n2)')
ax[1].set_ylabel('error')
fig.tight_layout()
plt.show()